In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/odev2/ieee-fraud-detection.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/drive/MyDrive/odev2")

In [ ]:
train_transaction = pd.read_csv("/content/drive/MyDrive/odev2/train_transaction.csv")
train_identity = pd.read_csv("/content/drive/MyDrive/odev2/train_identity.csv")
test_transaction = pd.read_csv("/content/drive/MyDrive/odev2/test_transaction.csv")
test_identity = pd.read_csv("/content/drive/MyDrive/odev2/test_identity.csv")

# Preprocessing

In [ ]:
train = train_transaction.merge(train_identity, on="TransactionID", how="left") #train ve test datalarımı olusturuyorum. Train ve test datalari iki parca halinde verilmis o yuzden birlestiriyorum
del train_transaction
del train_identity
test = test_transaction.merge(test_identity, on="TransactionID", how="left")
del test_transaction
del test_identity

In [ ]:
train.head(10)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
5,2987005,0,86510,49.0,W,5937,555.0,150.0,visa,226.0,debit,272.0,87.0,36.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2987006,0,86522,159.0,W,12308,360.0,150.0,visa,166.0,debit,126.0,87.0,0.0,NaN,yahoo.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2987007,0,86529,422.5,W,12695,490.0,150.0,visa,226.0,debit,325.0,87.0,NaN,NaN,mail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2987008,0,86535,15.0,H,2803,100.0,150.0,visa,226.0,debit,337.0,87.0,NaN,NaN,anonymous.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
9,2987009,0,86536,117.0,W,17399,111.0,150.0,mastercard,224.0,debit,204.0,87.0,19.0,NaN,yahoo.com,NaN,2.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,1.0,0.0,12.0,2.0,61.0,61.0,30.0,318.0,30.0

In [ ]:
train.isnull().sum()

TransactionID          0
isFraud                0
TransactionDT          0
TransactionAmt         0
ProductCD              0
                   ...  
id_36             449555
id_37             449555
id_38             449555
DeviceType        449730
DeviceInfo        471874
Length: 434, dtype: int64

In [ ]:
train.shape

(590540, 434)

In [ ]:
y = train['isFraud'].copy() #Sonraki islemlerde isFraud sutununa bisey olmasin diye o sutunu y degiskeninin icine kopyalıyoruz.
train = train.drop('isFraud', axis = 1) #Daha sonra datasetten o sutunu cikariyoruz
# save numpy array as npy file
from numpy import asarray
from numpy import save
# save to npy file
save('/content/drive/MyDrive/odev2/y_train.npy', np.array(y))  #sonradan kullanabilmek icin npy olarak kaydediyorum

In [ ]:
vals = []     # Her bir sutunun % kacinin null oldugu bilgisini tasiyan bir liste olusturuyoruz
for i in train:
    if train[i].isnull().sum() != 0:
        vals.append(train[i].isnull().sum() / train.shape[0])
vals = np.array(vals)
print(vals.mean())
print(vals.std())

0.4725187779497321
0.3596947876969992


In [ ]:
columns = []  # columns adli bir degisken olusturuyoruz. Bunun icine sutunlarda %35ten fazlasi null(NaN) olan sutunlari atiyoruz. (Cok fazla 0 oldugu icin bu sutunlari silecegiz)
for i,j in enumerate(train):
    if train[j].isnull().sum()/train.shape[0] > 0.35:
        columns.append(j)
print(columns)
print(len(columns))
print(train.shape[1])

['dist1', 'dist2', 'R_emaildomain', 'D2', 'D3', 'D5', 'D6', 'D7', 'D8', 'D9', 'D11', 'D12', 'D13', 'D14', 'M1', 'M2', 'M3', 'M4', 'M5', 'M7', 'M8', 'M9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144', 'V145', 'V146', 'V147', 'V148', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 'V156', 'V157', 'V158', 'V159', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V168', 'V169', 'V170', 'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 'V178', 'V179', 'V180', 'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 'V187', 'V188', 'V189', 'V190', 'V191', 'V192', 'V193', 'V194', 'V195', 'V196', 'V197', 'V198', 'V199', 'V200', 'V201', 'V202', 'V203', 'V204', 'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V211', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V218', 'V219', 'V220', 'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V230', 'V231', 'V232', 'V233', 'V234', 'V

In [ ]:
train = train.drop(columns, axis=1) # %35 ten fazla 0 olan sutunlari siliyoruz

In [ ]:

for i in range(len(columns)):   #train datasinda bazi yerler "_" ile verilmis ama test datasinda "-" olarak verilmis. Train datasinda o sutunlari sildigimiz icin test datasinda da silmemiz gerek.
    if "id" in columns[i] and "_" in columns[i]:  # Bu yuzden onlari degistirmemiz gerekiyor.
        columns[i] = columns[i].replace("_", "-")

In [ ]:
test = test.drop(columns, axis = 1)  # Burada o sutunlari test datasindan da atiyoruz

In [ ]:
print(test.shape) # en son train ve test datamiz ne hale gelmis gormek icin yazdik (satir sayisi yani veri sayisi , sutun sayisi yani parametre sayisi)
print(train.shape)

(506691, 201)
(590540, 201)


In [ ]:
all_data = pd.concat([train, test]) # tum datayi birlestiriyoruz
del train
del test

In [ ]:
all_data.shape  # Tum datanin sekli

(1097231, 201)

In [ ]:
all_data.isnull().sum().sum() # Tum datada kac tane 0 olduguna bakmak istedik. Gerekli degil

11805499

In [ ]:
all_data.drop('TransactionID', axis=1, inplace=True) # Transaction id sirali bir sekilde artan bir parametre. Hicbir matematiksel anlami yok o yuzden cikarmamiz gerekiyor

In [ ]:
counter = 0    #Kac sutunda en az 1 null var gormek istedik o yuzden baktik
for i in all_data:
    if all_data[i].isnull().sum() != 0:
        counter += 1
print(counter)

196


In [ ]:
for i,j in enumerate(all_data):   # null olan degerleri bir sekilde doldurmamiz lazim. Bunun icin en basit yontem o sutunun mean'ini koymak olurdu fakat biz biraz daha iyi sonuca alabilmek icin Quantile islemi uyguladik.
    if all_data[j].isnull().sum() >0 and j not in ['card4', 'card6', 'P_emaildomain', 'M6', 'ProductCD']: #
        vals = np.array([value for value in all_data[j].values if not np.isnan(value)])
        Q1 = np.quantile(vals,0.25)   #Q1 quantile'ini aldik
        Q3 = np.quantile(vals,0.75)   #Q3 quantile'ini aldik
        range = (Q3 - Q1)*1.5         #Quantile hesabi icin 1.5 ile carpip Q3'e ekleyerek , Q1'den ise cikararak islem yapmamiz lazim.
        clean = np.array([value for value in vals if value <= Q3+range and value >= Q1-range ]) # Outlier'lari cikardik
        if clean.size > 0:               # Outlierlari cikarinca kalan dizinin boyutu 0'dan buyuk ise
            if clean.std()/clean.max() - clean.min() > 0.45:    #Eger standart sapma %45 ten buyuk ise bosluklari median ile doldur. Dusuk ise bosluklari mean ile doldur
                fill = np.median(clean)   
                all_data[j].fillna(fill ,inplace = True)  # Bos degerleri median ile doldur
            else:
                all_data[j].fillna(clean.mean() ,inplace = True)  # Bos degerleri mean ile doldur
del range

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [ ]:
all_data['card4'] = all_data['card4'].fillna(all_data['card4'].mode()[0])   # Kategorik sutunlardaki bosluklari moduyla doldurduk.
all_data['card6'] = all_data['card6'].fillna(all_data['card6'].mode()[0])
all_data['M6'] = all_data['M6'].fillna(all_data['M6'].mode()[0])
all_data['P_emaildomain'] = all_data['P_emaildomain'].fillna(all_data['P_emaildomain'].mode()[0])
all_data['ProductCD'] = all_data['ProductCD'].fillna(all_data['ProductCD'].mode()[0])

In [ ]:
all_data.isnull().sum()

TransactionDT     0
TransactionAmt    0
ProductCD         0
card1             0
card2             0
                 ..
V317              0
V318              0
V319              0
V320              0
V321              0
Length: 200, dtype: int64

In [ ]:
from sklearn.preprocessing import StandardScaler  # Butun veriyi ayni aralik olacak sekilde olceklendirdik
sc = StandardScaler()
for i,j in enumerate(all_data):
    if all_data[j].dtype != 'O':
        column = all_data[j].values.copy()
        column = sc.fit_transform(column.reshape(column.shape[0],1))
        all_data[j] = column.copy()

In [ ]:
arr = ['card4', 'card6', 'P_emaildomain', 'M6', 'ProductCD']  #One hot encoder ile sutunu duzenledik. Daha sonra bu sutunlari sildik
for i in arr:
    dummies = pd.get_dummies(all_data[i])
    all_data = pd.concat([all_data,dummies], axis = 1)
    all_data.drop(i, axis = 1, inplace = True)

In [ ]:
#all_data.to_pickle('/content/drive/MyDrive/odev2/anil.pkl', protocol=4)    #bu islemler sonucunda cikan all_data 'yi daha sonra kolayca kullanabilmek icin yukledik
del train     # train ve test datasina ihtiyacimiz kalmadi silebiliriz
del test

In [ ]:
from numpy import load
y = load('/content/drive/MyDrive/odev2/y_train.npy')   # y ve all_data'yı daha sonradan projeye dondugumuzde kolayca alabilmek icin
all_data = pd.read_pickle('/content/drive/MyDrive/odev2/anil.pkl')  

In [ ]:
train = all_data.iloc[:590540, :] # train ve test datamizi basta belirtilen sekilde ikiye ayirdik
test = all_data.iloc[590540:, :]

In [ ]:
from sklearn.model_selection import train_test_split    # datamizi X_train,X_test,y_train,y_test olarak ayirdik artik model olusturmaya gecebiliriz. Datamiz hazir.
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3)

In [ ]:
y_train

264917    0
322697    0
244659    0
66589     0
288658    0
         ..
420736    0
547614    0
433710    0
401453    0
402466    0
Name: isFraud, Length: 413378, dtype: int64

# Models

# Random Forest
F1 Score: 0.61

In [ ]:
from sklearn.ensemble import RandomForestClassifier   #Modelleri olusturmak icin gerekli importlar
from sklearn.naive_bayes import GaussianNB
import tensorflow as tf 
from tensorflow import keras 
from keras.models import Sequential 
from keras.layers import Dense,Dropout,LeakyReLU
from keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score, roc_auc_score

In [ ]:
model=RandomForestClassifier( # Modelin daha iyi olmasi icin daha iyi preprocessing yapilabilir. Parametre optimizasyonu yapilabilir. Bu modelin adi RandomForest
    n_estimators= 300,
    max_depth=50,
    criterion='entropy',
    max_features='sqrt'
)
model.fit(X_train,y_train)    # modelin egitilmesi
acc = model.score(X_test,y_test)
pred = model.predict(X_test)

In [ ]:
f1 = f1_score(y_test, pred)   # Random forestın sonucu burada daha cok bizi F1 score ilgilendiriyor
print('Accuracy:',acc )
print('f1 score:',f1)

Accuracy: 0.9804021178356532
f1 score: 0.61753690240141


# Neural Network (5 Hidden Layer)
F1 Score: 0.65

In [ ]:
nn = Sequential()   # Neural Network Yapisi ile ikinci model

nn.add(Dense(2048, activation='relu'))
nn.add(Dropout(0.3))

nn.add(Dense(1024, activation='relu'))
nn.add(Dropout(0.3))

nn.add(Dense(512, activation='relu'))
nn.add(Dropout(0.3))

nn.add(Dense(256, activation='relu'))
nn.add(Dropout(0.3))

nn.add(Dense(128, activation='relu'))
nn.add(Dropout(0.3))

nn.add(Dense(1, activation='sigmoid'))

nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
nn.fit(X_train, y_train, batch_size=5000, epochs=100,callbacks=[EarlyStopping(monitor='val_loss', patience=5)], validation_data=(X_test,y_test)) # modelin egitilmesi

Epoch 1/100
83/83 [==============================] - 6s 47ms/step - loss: 0.1854 - accuracy: 0.9532 - val_loss: 0.1232 - val_accuracy: 0.9705
Epoch 2/100
83/83 [==============================] - 3s 37ms/step - loss: 0.1125 - accuracy: 0.9696 - val_loss: 0.1057 - val_accuracy: 0.9726
Epoch 3/100
83/83 [==============================] - 3s 37ms/step - loss: 0.1041 - accuracy: 0.9717 - val_loss: 0.0973 - val_accuracy: 0.9731
Epoch 4/100
83/83 [==============================] - 3s 37ms/step - loss: 0.0984 - accuracy: 0.9729 - val_loss: 0.0980 - val_accuracy: 0.9742
Epoch 5/100
83/83 [==============================] - 3s 37ms/step - loss: 0.0943 - accuracy: 0.9739 - val_loss: 0.0971 - val_accuracy: 0.9752
Epoch 6/100
83/83 [==============================] - 3s 37ms/step - loss: 0.0903 - accuracy: 0.9750 - val_loss: 0.0892 - val_accuracy: 0.9755
Epoch 7/100
83/83 [==============================] - 3s 37ms/step - loss: 0.0876 - accuracy: 0.9759 - val_loss: 0.0932 - val_accuracy: 0.9759
Epoch 

In [ ]:
pred = list(map(lambda x: 1 if x > 0.5 else 0 ,nn.predict(X_test))) #neural networkun sonucları
f1 = f1_score(y_test, pred)
print('f1 score:',f1)

f1 score: 0.6587088431314035


In [ ]:
from numpy import loadtxt
from sklearn.metrics import accuracy_score
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
import xgboost as xgb
from xgboost import XGBClassifier
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import time

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be import

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.3)

# XgBoost Default

1.   Time: 3min 36s
2.   Kaggle Score : 0.683979



In [ ]:
%%time
modelXG=XGBClassifier()
modelXG.fit(X_train,y_train)

print("Start Predicting")
predictions = modelXG.predict(X_test)
pred_proba = modelXG.predict_proba(X_test)[:, 1]

print("\nResulting Metrics")
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, predictions))
print("AUC : %f" % metrics.roc_auc_score(y_test, pred_proba))
print("F1 Score : %f" % metrics.f1_score(y_test, predictions))


Start Predicting

Resulting Metrics
Accuracy : 0.9729
AUC : 0.887473
F1 Score : 0.400250
CPU times: user 3min 35s, sys: 1.29 s, total: 3min 36s
Wall time: 3min 36s


In [ ]:
pred = modelXG.predict(test)    #0.683979
pred = pd.DataFrame(pred, columns=['isFraud'])
t_id = [i for i in range(3663549,4170240)]
pred["TransactionID"] = t_id
pred.set_index("TransactionID", inplace=True)
pred.to_csv("/content/drive/MyDrive/odev2/xgBoost_default2.csv")

In [ ]:
!pip install catboost

     |████████████████████████████████| 65.8MB 47kB/s 


In [ ]:
from catboost import Pool, CatBoostClassifier, cv

# CatBoost Default


1.   Time: 18.5s
2.   Kaggle Score : 0.715378



In [ ]:
%%time
#model = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42)
model = CatBoostClassifier(task_type='GPU',eval_metric='Accuracy',random_seed=42)

#now just to make the model to fit the data
model.fit(X_train,y_train,eval_set=(X_test,y_test))

In [ ]:
print("Start Predicting")
predictions = model.predict(X_test)
pred_proba = model.predict_proba(X_test)[:, 1]

print("\nResulting Metrics")
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, predictions))
print("AUC : %f" % metrics.roc_auc_score(y_test, pred_proba))
print("F1 Score : %f" % metrics.f1_score(y_test, predictions))

Start Predicting

Resulting Metrics
Accuracy : 0.9781
AUC : 0.913500
F1 Score : 0.565378


In [ ]:
pred = model.predict(test)    # score = 0.715378
pred = pd.DataFrame(pred, columns=['isFraud'])
t_id = [i for i in range(3663549,4170240)]
pred["TransactionID"] = t_id
pred.set_index("TransactionID", inplace=True)
pred.to_csv("/content/drive/MyDrive/odev2/catboost_default3.csv")

# Hyperparameter Optimization

# Random Search For CatBoost

1.   Kaggle Score= 0.710191
2.   Time: 68s



In [ ]:

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt  

parameters = {'depth'         : sp_randInt(1, 10),
              'learning_rate' : sp_randFloat(),
              'iterations'    : sp_randInt(10, 100)
}
    
randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                               cv = 2, n_iter = 10)
randm.fit(X_train, y_train, eval_set=(X_test,y_test))

In [ ]:
randm.best_estimator_

In [ ]:
randm.best_params_

{'depth': 9, 'iterations': 47, 'learning_rate': 0.36728949846879244}

In [ ]:
%%time
print("Start Predicting")
predictions = randm.best_estimator_.predict(X_test)
pred_proba = randm.best_estimator_.predict_proba(X_test)[:, 1]

print("\nResulting Metrics")
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, predictions))
print("AUC : %f" % metrics.roc_auc_score(y_test, pred_proba))
print("F1 Score : %f" % metrics.f1_score(y_test, predictions))

Start Predicting

Resulting Metrics
Accuracy : 0.9775
AUC : 0.910361
F1 Score : 0.552909
CPU times: user 512 ms, sys: 2.78 ms, total: 515 ms
Wall time: 378 ms


In [ ]:
pred = randm.predict(test)    # score = 0.710191
pred = pd.DataFrame(pred, columns=['isFraud'])
t_id = [i for i in range(3663549,4170240)]
pred["TransactionID"] = t_id
pred.set_index("TransactionID", inplace=True)
pred.to_csv("/content/drive/MyDrive/odev2/catboost_random3.csv")

In [ ]:
!pip install scikit-optimize

In [ ]:
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

# Bayesian Optimization for CatBoost 
1.   Kaggle Score= 0.726318
2.   Time = 72 sec

In [ ]:
roc_auc = make_scorer(roc_auc_score, greater_is_better=True, needs_threshold=True)

# Defining your search space
search_spaces = {'iterations': Integer(10, 300),
                 'depth': Integer(1, 8),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 #'random_strength': Real(1e-9, 10, 'log-uniform'),
                 #'bagging_temperature': Real(0.0, 1.0),
                 #'border_count': Integer(1, 255),
                 #'l2_leaf_reg': Integer(2, 30),
                 #'scale_pos_weight':Real(0.01, 1.0, 'uniform')
                 }
# Setting up BayesSearchCV
opt = BayesSearchCV(model,
                    search_spaces,
                    scoring=roc_auc,
                    n_iter=2,
                    n_jobs=-1,  
                    return_train_score=False,
                    refit=True, #use best model
                    optimizer_kwargs={'base_estimator': 'GP'},
                    random_state=42)
# Running the optimization
opt.fit(X_train, y_train)

0:	learn: 0.9680607	total: 17.4ms	remaining: 3.82s
1:	learn: 0.9683897	total: 34.5ms	remaining: 3.78s
2:	learn: 0.9691251	total: 51.2ms	remaining: 3.72s
3:	learn: 0.9702887	total: 80.2ms	remaining: 4.35s
4:	learn: 0.9707459	total: 96.5ms	remaining: 4.17s
5:	learn: 0.9711910	total: 112ms	remaining: 4.02s
6:	learn: 0.9717039	total: 130ms	remaining: 3.97s
7:	learn: 0.9719095	total: 147ms	remaining: 3.9s
8:	learn: 0.9723111	total: 163ms	remaining: 3.83s
9:	learn: 0.9723401	total: 179ms	remaining: 3.77s
10:	learn: 0.9725796	total: 195ms	remaining: 3.73s
11:	learn: 0.9726642	total: 211ms	remaining: 3.67s
12:	learn: 0.9726884	total: 232ms	remaining: 3.71s
13:	learn: 0.9727949	total: 249ms	remaining: 3.67s
14:	learn: 0.9728795	total: 265ms	remaining: 3.63s
15:	learn: 0.9730150	total: 278ms	remaining: 3.56s
16:	learn: 0.9732013	total: 292ms	remaining: 3.51s
17:	learn: 0.9733368	total: 305ms	remaining: 3.44s
18:	learn: 0.9734214	total: 318ms	remaining: 3.39s
19:	learn: 0.9735738	total: 333ms	rem

BayesSearchCV(cv=None, error_score='raise',
              estimator=<catboost.core.CatBoostClassifier object at 0x7fbc1c34f048>,
              fit_params=None, iid=True, n_iter=2, n_jobs=-1, n_points=1,
              optimizer_kwargs={'base_estimator': 'GP'},
              pre_dispatch='2*n_jobs', random_state=42, refit=True,
              return_train_score=False,
              scoring=make_scorer(roc_auc_score, needs_threshold=True),
              search_spaces={'depth': Integer(low=1, high=8, prior='uniform', transform='identity'),
                             'iterations': Integer(low=10, high=300, prior='uniform', transform='identity'),
                             'learning_rate': Real(low=0.01, high=1.0, prior='log-uniform', transform='identity')},
              verbose=0)

In [ ]:
print("Start Predicting")
predictions = opt.best_estimator_.predict(X_test)
pred_proba = opt.best_estimator_.predict_proba(X_test)[:, 1]

print("\nResulting Metrics")
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, predictions))
print("AUC : %f" % metrics.roc_auc_score(y_test, pred_proba))
print("F1 Score : %f" % metrics.f1_score(y_test, predictions))

Start Predicting

Resulting Metrics
Accuracy : 0.9781
AUC : 0.906868
F1 Score : 0.579016


In [ ]:
pred = opt.predict(test)    #0.726318
pred = pd.DataFrame(pred, columns=['isFraud'])
t_id = [i for i in range(3663549,4170240)]
pred["TransactionID"] = t_id
pred.set_index("TransactionID", inplace=True)
pred.to_csv("/content/drive/MyDrive/odev2/catboost_Bayes3.csv")

# Bayesian Optimization for XgBoost


1.   Kaggle Score:0.730195
2.   Time:37 min



In [ ]:
roc_auc = make_scorer(roc_auc_score, greater_is_better=True, needs_threshold=True)

# Defining your search space
search_spaces = {'iterations': Integer(10, 300),
                 'depth': Integer(1, 8),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 #'random_strength': Real(1e-9, 10, 'log-uniform'),
                 #'bagging_temperature': Real(0.0, 1.0),
                 #'border_count': Integer(1, 255),
                 #'l2_leaf_reg': Integer(2, 30),
                 #'scale_pos_weight':Real(0.01, 1.0, 'uniform')
                 }
# Setting up BayesSearchCV
opt = BayesSearchCV(modelXG,
                    search_spaces,
                    scoring=roc_auc,
                    n_iter=2, 
                    return_train_score=False,
                    refit=True, #use best model
                    optimizer_kwargs={'base_estimator': 'GP'},
                    random_state=42)
# Running the optimization
opt.fit(X_train, y_train)

BayesSearchCV(cv=None, error_score='raise',
              estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                      colsample_bylevel=1, colsample_bynode=1,
                                      colsample_bytree=1, gamma=0,
                                      learning_rate=0.1, max_delta_step=0,
                                      max_depth=3, min_child_weight=1,
                                      missing=None, n_estimators=100, n_jobs=1,
                                      nthread=None, objective='binary:logistic',
                                      random_state=0, reg_alpha=0, reg_lambda=1,...
              pre_dispatch='2*n_jobs', random_state=42, refit=True,
              return_train_score=False,
              scoring=make_scorer(roc_auc_score, needs_threshold=True),
              search_spaces={'depth': Integer(low=1, high=8, prior='uniform', transform='identity'),
                             'iterations': Integer(low=10, high=30

In [ ]:
opt.best_params_

OrderedDict([('depth', 4),
             ('iterations', 221),
             ('learning_rate', 0.7340675018434776)])

In [ ]:
print("Start Predicting")
predictions = opt.best_estimator_.predict(X_test)
pred_proba = opt.best_estimator_.predict_proba(X_test)[:, 1]

print("\nResulting Metrics")
print("Accuracy : %.4g" % metrics.accuracy_score(y_test, predictions))
print("AUC : %f" % metrics.roc_auc_score(y_test, pred_proba))
print("F1 Score : %f" % metrics.f1_score(y_test, predictions))

Start Predicting

Resulting Metrics
Accuracy : 0.9767
AUC : 0.910253
F1 Score : 0.554644


In [ ]:
pred = opt.predict(test)    
pred = pd.DataFrame(pred, columns=['isFraud'])
t_id = [i for i in range(3663549,4170240)]
pred["TransactionID"] = t_id
pred.set_index("TransactionID", inplace=True)
pred.to_csv("/content/drive/MyDrive/odev2/xgBoost_Bayes3.csv")

# Summary (CatBoost vs XgBoost)
CatBoost is much faster than XgBoost (almost x30). CatBoost's kaggle score is better then XgBoost's kaggle score for default parameters. 